In [42]:
#Import libraries for plotting, sampling
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

## Binance Take Home Test - Question 2

*We are all very familiar with Logistic Regression and its optimization solution to minimize its cost function. But in reality, we may need some constraints. The following are two special constraints, please give: a) optimization solution; b) code of the solution process; c) sample code (you can use the sklearn data set or generate your own code)*<br/><br/>
*1) Non-negative constraint: All coefficients of LR are required to be non-negative; <br/>
2) Order preserving constraint: The coefficient of LR is required to satisfy a1 >= a2 >= a3 …… <br/>
One more question: If there is no constraint, is LR a global optimization algorithm or a local one? why? When constraints 1) or 2) are imposed, global or local? why?*

In [43]:
X, y = load_breast_cancer(return_X_y=True) #Load breast cancer dataset (binary classification)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

mean = np.mean(X_train, axis=0) #Mean & std calculated using training set only
std = np.std(X_train, axis=0)

X_train = (X_train-mean)/std #Both train, test set normalised by training set mean/std to avoid leakage
X_test = (X_test-mean)/std

#Append a constant 1 to X in the train, test set for intercept

In [134]:
def prob(X, theta):
    """Returns probability of class 1 for an (N+1)xd dataset x"""
    return np.exp(np.matmul(X, theta.T)) / (1+np.exp(np.matmul(X, theta.T)))

def log_likelihood(X, y, theta):
    """Calculates log-likelihood for a dataset X, y using logistic regression w/parameters theta"""
    likelihoods = y*np.matmul(X, theta.T) - np.log(1+np.exp(1+np.exp(np.matmul(X, theta.T))))
    return likelihoods.sum()

def grad_log_likelihood(X, y, theta):

SyntaxError: unexpected EOF while parsing (1628588409.py, line 10)

In [129]:
prob(X_train[:,:3],np.array([0.5,0.3,0.5])).shape

(455,)

In [132]:
log_likelihood(X_train[:,:3], y_train, np.array([0.5,0.3,0.5]))

array([ -2.78558128,  -2.42209427,  -2.12856244,  -2.37361592,
        -2.20505283,  -2.36055928,  -2.64384207,  -2.31769708,
        -2.50985826,  -2.76787767, -10.762299  ,  -2.42334736,
        -2.22743718,  -2.77602223,  -2.79275858,  -2.44272372,
        -4.33559383,  -2.93222335,  -2.3041177 ,  -2.12647406,
        -2.48334097,  -2.85559499,  -2.28042849,  -2.36804678,
       -23.70168504,  -2.53403429,  -2.60403548,  -2.43258629,
        -5.50189379,  -2.44610193,  -2.43985228, -11.17251052,
        -2.19644847,  -3.17083325,  -2.35306391,  -2.20793669,
        -2.25590921,  -2.1226214 ,  -3.00714151,  -2.17255824,
        -2.84378516,  -2.80458005,  -2.19563465,  -2.37540568,
        -2.30308813,  -2.67445955,  -6.78326927,  -2.3164185 ,
        -2.59439484,  -4.76453571,  -2.62161152,  -3.38571458,
        -2.15453463,  -4.30959194,  -2.15650773,  -2.13873939,
        -2.70476653,  -2.64339214,  -2.64199525,  -2.35825794,
        -4.03496448,  -3.79134703,  -2.71705387,  -2.30

In [109]:
x = X[:,:3]
theta = [0.5,0.3,0.5]
np.exp(np.matmul(x,theta)).shape

(569,)

In [120]:
A = np.array([1,2,3])
B = np.array([4,5,6])
A*B

array([ 4, 10, 18])

In [128]:
X_train[:,:3]

array([[-1.15036482, -0.39064196, -1.12855021],
       [-0.93798972,  0.68051405, -0.94820146],
       [ 0.574121  , -1.03333557,  0.51394098],
       ...,
       [-1.32422924, -0.20048168, -1.31754581],
       [-1.24380987, -0.2245526 , -1.28007609],
       [-0.73694129,  1.14989702, -0.71226578]])